# 9.1 객체 표현 
- repr() : 객체를 개발자가 보고자 하는 형태로 표현한 문자열로 반환한다. \_\_repr__() 특별 메서드를 구현해야 한다.
- str()  : 객체를 사용자가 보고자 하는 형태로 표현한 문자열로 반환한다. \_\_str__() 특별 메서드를 구현해야 한다.
- bytes() : \_\_bytes__() 를 호출하여 객체를 바이트 시퀀스로 표현한다.
- \_\_format__() : 내장함수 format() 과 str.format() 메서드 둘다 사용하며, 특별 포맷 코드를 이용해서 객체를 표현하는 문자열을 반환한다.

# 9.2 벡터 클래스의 부활

In [38]:
# vector2d_v0.py 
from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y) # 미리 실수로 변환하는 센스
        
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

# 9.3 대안 생성자

```python
@classmethod
def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode) # 이렇게 형변환하면 memv = (3,4) 라고함. 그래서 *memv 로 사용된 것
    return cls(*memv)
```

# 9.4 @classmethod 와 @staticmethod
- @classmethod 데커레이터는 객체가 아닌 클래스에 연산을 수행하는 메서드를 정의
    - @classmethod 데커레이터는 파이썬의 고유한 기능이다.
    - @classmethod 는 메서드가 호출되는 방식을 변경해서 클래스 자체를 첫 번째 인수로 받게 만듦.
    - 예제 9.3 과 같이 frombytes() 와 같은 대안 생성자를 구현하기 위해 주로 사용됨
- @staticmethod
    - @staticmethod 데커레이터는 메서드가 특별한 첫 번째 인수를 받지 않도록 메서드를 변경한다.
    - 본질적으로 정적 메서드는 모듈 대신 클래스 본체 안에 정의된 평범한 함수일 뿐이다.
    - 저자는 클래스와 함께 작동하지 않는 함수를 정의하려면, 단지 함수를 모듈에 정의하면 된다고 말한다.

In [1]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

In [2]:
Demo.klassmeth()

(__main__.Demo,)

In [3]:
Demo.klassmeth("spam")

(__main__.Demo, 'spam')

In [4]:
Demo.statmeth()

()

# 9.5 포맷된 출력
- format() 내장 함수와 str.format() 메서드는 실제 포맷 작업을 \_\_format_\_(format_spec) 메서드에 위임한다.
- format_spec 은 포맷 명시자로서, 다음 두 가지 방법 중 하나를 통해 지정한다.
    - format(my_obj, format_spec) 의 두 번째 인수
    - str.format() 에 사용된 포맷 문자열 안에 {} 로 구분한 대체 필드안에서 콜론 뒤의 문자열
- 몇몇 내장 자요형은 포맹 명시 간이 언어에 자신만의 고유한 표현 코드를 가지고 있다.
    - int 경우 이진수를 나타내는 'b'
    - int 경우 16진수를 나타내는 'x'
    - float 형의 경우 고정 소수점을 나타내는 'f'
    - float 형의 경우 백분율을 나타내는 '%'

In [5]:
br1 = 1/2.43
br1

0.4115226337448559

In [6]:
format( br1, '0.4f')

'0.4115'

In [7]:
'1 BRL = {rate:0.2f} USD'.format(rate = br1)

'1 BRL = 0.41 USD'

#### 각 클래스가 format_spec 인수를 자신이 원하는 대로 해석해서 포맷 명시 간이 언어를 확장할 수 있다.

In [8]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'14:46:14'

In [9]:
"It's now {:%I:%M %p}".format(now)

"It's now 02:46 PM"

클래스에서 \_\_format_\_() 메서드를 정의하지 않으면, object 에서 상속받은 메서드가 str(my_object) 를 반환한다.
- Vector2d 는 \_\_str()\_\_() 을 정의하고 있으므로 다음과 같이 실행됨

In [13]:
v1 = Vector2d(3,4)
format(v1)

'(3.0, 4.0)'

In [15]:
def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self)
    return '({},{})'.format(*components)

In [16]:
def angle(self):
    return math.atan2(self.y, self.x)

In [17]:
def __format__(self, fmt_spec =''):
    if fmt_spec.endswith('p'):
        fmt_spec = fmt_spec[:-1]
        coords = (abs(self), self.angle())
        outer_fmt = '<{},{}>'
    
    else:
        coords = self
        outer_fmt = '({},{})'
        
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)

# 9.6 해시 가능한 Vector2d
- 해시 가능하게 만들려면 \_\_hash_\_() 메서드를 구현해야한다. ( \_\_eq_\_() 메서드도 필요하지만 이미 구현함 )
- @property 데커레이터는 프로퍼티의 getter method 를 나타낸다.
- 정확히 두 개의 언더바로 시작해서 ( 뒷부분의 언더바는 없거나 하나만 사용한다 ) 속성을 비공개로 만든다.

In [19]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

In [22]:
v1 = Vector2d(3,4)
v2 = Vector2d(3.1,4.2)
print(hash(v1))
print(hash(v2))

7
384307168202284039


In [23]:
set([v1,v2])

{<__main__.Vector2d at 0x2073dc91310>, <__main__.Vector2d at 0x2073dc91880>}

# 9.7 파이썬에서의 비공개 속성과 보호된 속성
- 서브 클래스에서 '비공개'성격의 속성을 실수로 변경하지 못하게 하는 간단한 메커니즘이 있다.
- 속성명을 \_\_mood 처럼 두 개의 언더바로 시작하고 언더바 없이 또는 하나의 언더바로 끝나도록 정의하면, 파이썬은 언더바와 클래스명을 변수명 앞에 붙여 객체의 \_\_dict_\_ 에 저장한다. 따라서 Dog 클래스의 경우 \_\_mood 는 \_Dog\_\_mood가 된다. 이러한 파이썬 언어 기능을 <b>_이름 장식_</b>이라고 한다.
- 이름 장식은 안전을 제공하지만, 보안 기느은 아니다. 실수로 접근하는 것은 막도록 설계되어 있지만 고의적인 악용을 막지 못한다.
    - Vector2d 의 비공개 요소에 직접 갑슬 할당할 수 있기 때문이다.
- self.\_x 처럼 언더바 하나를 앞에 붙여 속성을 '보호'하는 것을 좋아하는 개발자도 있다.
    - 속성명 앞에 언더바 하나를 붙이더라도 파이썬 인터프리터가 별도로 특별히 처리하는 것은 없다.
    - 그러나 클래스 외부에서 그런 속성에 접근하지 않는 것은 파이썬 프로그래머 사이에서 일종의 금기처럼 자리 잡혀있다.

In [24]:
v1 = Vector2d(3,4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [25]:
v1._Vector2d__x

3.0

# 9.8 \_\_slots__ 클래스 속성으로 공간 절약하기
- 기본적으로 파이썬은 객체 속성을 각 객체안의 \_\_dict__ 라는 딕셔너리형 속성에 저장한다.
- 딕셔너리는 빠른 접근 속도를 제공하기 위해 내부에 해시 테이블을 유지하므로 메모리 사용량 부담히 상당히 크다.
    - 고정된 스키마의 아주 큰 데이터베이스와 같은 테이블 형태의 테이터를 사용할 때 유용하다.
- 만약 속성이 몇 개 없는 수백만 개의 객체를 다룬다면, \_\_slots__ 클래스 속성을 이용해서 메모리 사용량을 엄청나게 줄일 수 있다.
- \_\_slots__ 속성은 파이썬 인터프리터가 객체 속성을 딕셔너리 대신 튜플에 저장하게 만든다.
- \_\_slots__를 클래스에 정의함으로써 '이 속성들이 이 클래스 객체가 가지는 속성'임을 인터프리터에 알려준다. 그러면 파이썬 인터프리터는 이 속성들을 각 객체의 튜플형 구조체에 저장함으로써 \_\_dict__ 속성을 각 객체마다 유지하는 부담을 덜어낸다

In [43]:
# vector2d_slots_v3.py

from array import array
import math

class Vector2d:
    
    __slots__  = ('__x','__y')

    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y) # 미리 실수로 변환하는 센스
    
    @property # 게터 메서드를 나타냄
    def x(self):
        return self.__x        
        
    @property
    def y(self):
        return self.__y

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)        
    
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '<{}, {}>'            
        components = (format(c, fmt_spec) for c in coords)
        return '({}, {})'.format(*components)

    @classmethod # 클래스 메서드
    def frombytes(cls, octets): # self 매개변수가 없고 대신 자신이 cls로 전달됨
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [30]:
class Vector2d:
    __slots__  = ('__x','__y')
    
    typecode = 'd'
    
    # 뒤의 메서드 생략

- 클래스안에 \_\_slots__ 를 명시하는 경우, 객체는 \_\_slots__에 명시되지 않은 속성을 가질 수 없게 된다. 이는 \_\_slots__가 존재하는 이유는 아니며, 실제는 부작용이다. 
- 그러나 단지 여러분이 만든 클래스의 사용자가 객체에 새로운 속성을 추가할 수 없게 하기 위해 \_\_slots__ 를 사용하는 것은 적절치 않다.

## 9.8.1 \_\_slots__를 사용할 때 주의할 점
- 인터프리터는 상속된 \_\_slots__ 속성을 무시하므로 각 클래스마다 \_\_slots__ 속성을 다시 정의해야한다.
- \_\_dict__ 를 \_\_slots__ 에 추가하지 않는 한 객체는 \_\_slots__에 나열된 속성만 가질 수 있다. ( 그러나 \_\_dict__ 를 \_\_slots__ 에 추가 하면 메모리 절감 효과가 반감될 수 있다. ) 
- \_\_weakref__ 를 \_\_slots__에 추가하지 않으면 객체가 약한 참조의 대상이 될 수 없다.

- 프로그램이 수백만 개의 객체를 다루는 경우가 아니라면, 굳이 동적 속성을 받아들이지 않고 약한 참조를 지원하지도 않는 까다로운 클래스를 만든 필요가 없다.

# 9.9 클래스 속성 오버라이드
- 클래스 속성을 객체 속성의 기본값으로 사용하는 것은 파이썬의 독특한 특징이다.
- 그러나 존재하지 않는 객체 속성에 값을 저장하면, 새로운 객체 속성 ( e.g. typecode 객체 속성 ) 을 생성하고 동일한 이름의 클래스 속성은 변경하지 않는다. 그 후부터는 객체가 self.typecode 를 읽을 때 객체 자체의 typecode 를 가져오므로, 동일한 이름의 클래스 속성을 가리게된다. 그러면 각 객체가 서로 다른 typecode 를 갖도록 커스터마이즈할 수 있게 된다.
- 클래스의 속성을 변경하려면 클래스 정의에서 직접 바꿔야하며, 객체를 통해 변경하면 안된다. 다음과 같이 클래스 속성을 변경하면 모든 객체의 기본 typecode도 바뀐다.
```python
Vector2d.typecode = 'f
```
- 클래스 속성은 공개되어 있고, 모든 서브 클래스가 상속하므로, 클래스 데이터 속성을 커스터마이즈할 때는 클래스를 상속하는 것이 일반적인 방식이다.